In [1]:
import numpy as np

In [2]:
from src.lib.model import model
from src.lib.activation_functions import ReLu, softmax
from src.lib.layers import Dense, flatten, BatchNorm1D, Conv2D, MaxPooling2D
from src.lib.loss_functions import CrossEntropy
from src.lib.trainer import trainer
from src.lib.optimizers import Adam, SGD, AdamGPU

In [3]:

from tensorflow.keras.datasets import fashion_mnist

num_classes = 10
# Chargement
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normaliser
x_train = x_train / 255.0
x_test = x_test / 255.0

def one_hot_encode(y, num_classes):
    return np.eye(num_classes)[y]

# Utilisation
y_train_encoded = one_hot_encode(y_train, num_classes)
y_test_encoded = one_hot_encode(y_test, num_classes)

print(y_train_encoded.shape)  # (60000, 10)
print(y_test_encoded.shape)   # (10000, 10)


(60000, 10)
(10000, 10)


In [4]:
fashion_mnist_labels = {
    0: "T-shirt/top",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle boot"
}


In [5]:
x_train = x_train[:, np.newaxis, :, :]
x_test = x_test[:, np.newaxis, :, :]

In [11]:
from src.lib.backend import backend

backend.set_gpu(True)

np = backend.xp

print(np)


<module 'cupy' from 'c:\\Python311\\Lib\\site-packages\\cupy\\__init__.py'>


In [ ]:
#1mn07 GPU
#adamgpy 51.9s
#cpu 26.7s

#cpu 52.3s
#gpu 33.6s
#adamgpu 20.8s

In [14]:
optimizer = AdamGPU(learning_rate=0.001)

input_shape = x_train.shape

print(input_shape)

x_train = backend.asarray(x_train)
y_train_encoded = backend.asarray(y_train_encoded)
x_test = backend.asarray(x_test)
y_test_encoded = backend.asarray(y_test_encoded)

model_mnist = model([
    
    flatten(),
    Dense(512),
    BatchNorm1D(), 
    ReLu(), 

    
    Dense(128),
    BatchNorm1D(), 
    ReLu(), 
    
    Dense(10), 
    
    CrossEntropy(l2_lambda=0)
], 
input_shape=input_shape)

model_mnist.summary()


trainer_mnist = trainer(model_mnist, x_train, y_train_encoded, custom_train_test_set=(x_train, y_train_encoded, x_test, y_test_encoded))
# Dans une cellule Jupyter :
%prun -l 20 -s cumulative trainer_mnist.train(nb_epochs = 10, optimizer=optimizer, batch_size = 512)


(60000, 1, 28, 28)
┌──────────────────────────────────────────────────────────────────────┐
Layer (type)         Input Shape               Output Shape             
├────────────────────┬─────────────────────────┬─────────────────────────┤
flatten              (1, 28, 28)               (784,)                   
Dense                (784,)                    (512,)                   
BatchNorm1D          (512,)                    (512,)                   
ReLu                 (512,)                    (512,)                   
Dense                (512,)                    (128,)                   
BatchNorm1D          (128,)                    (128,)                   
ReLu                 (128,)                    (128,)                   
Dense                (128,)                    (10,)                    
CrossEntropy         (10,)                     10                       
└──────────────────────────────────────────────────────────────────────┘
0
2.3113215
0.1171
1
0.5529833

         398712 function calls in 20.703 seconds

   Ordered by: cumulative time
   List reduced from 148 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   20.703   20.703 {built-in method builtins.exec}
        1    0.000    0.000   20.703   20.703 <string>:1(<module>)
        1    0.022    0.022   20.703   20.703 trainer.py:59(train)
     1192    0.091    0.000    9.640    0.008 model.py:31(forward)
     1180    0.073    0.000    8.486    0.007 model.py:48(backward)
     2360    3.086    0.001    4.705    0.002 layers.py:113(backward)
     2384    2.828    0.001    4.692    0.002 layers.py:94(forward)
     1192    0.824    0.001    3.470    0.003 loss_functions.py:13(forward)
    17736    0.082    0.000    2.918    0.000 sumprod.py:11(sum)
    17736    2.806    0.000    2.806    0.000 {method 'sum' of 'cupy._core.core._ndarray_base' objects}
     1180    0.105    0.000    1.877    0.002 model.py:62(upd